In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

# XGB 历史数据

In [4]:
train = pd.read_csv('train_preprocessing42.csv')

test = train[train.is_trade == -1]
train = train[train.is_trade != -1]

In [5]:
predictors = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description',
    'item_category_split_count', 'item_property_split_count',
    'predict_category_split_count', 'user_shop_count', 'user_item_count',
    'user_context_count', 'user_shop_trade_count', 'user_item_trade_count',
    'user_context_trade_count', 'user_brand_count', 'user_city_count',
    'user_brand_trade_count', 'user_city_trade_count', 'item_occupation_count',
    'item_age_count', 'item_gender_count', 'item_occupation_trade_count',
    'item_age_trade_count', 'item_gender_trade_count', 'shop_score',
    'item_score', 'item_score2', 'item_score3', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%', 'pred_category_split_count',
    'pred_property_split_count'
]

target = ['is_trade']

In [6]:
train[predictors].shape
train[predictors].head()
test[predictors].shape
test[predictors].head()

(154611, 46)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,item_category_split_count,item_property_split_count,predict_category_split_count,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,pred_category_split_count,pred_property_split_count
0,3,3,4,14,1,1003,2005,3003,5,4,1.0,3,1.0,1.0,1.0,2,22,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,6.0,3.0,2.0,0.0,0.0,0.0,7.0,59,19.666667,3.277778,2,0,1.0,0.000000,0.400000,0.000000,5,1
1,3,3,4,14,0,1002,2005,3006,0,4,1.0,3,1.0,1.0,1.0,2,22,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,0.0,0.0,7.0,59,19.666667,19.666667,2,1,1.0,0.045455,1.000000,0.500000,2,2
2,3,3,4,14,0,1003,2005,3004,0,4,1.0,3,1.0,1.0,1.0,2,22,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,3.0,5.0,0.0,0.0,0.0,7.0,59,19.666667,19.666667,2,1,1.0,0.045455,0.666667,0.333333,3,3
3,3,3,4,14,1,1004,2005,3006,15,4,1.0,3,1.0,1.0,1.0,2,22,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,2.0,0.0,0.0,0.0,7.0,59,19.666667,1.229167,2,0,1.0,0.000000,0.400000,0.000000,5,3
4,3,3,4,14,0,1002,2005,3001,0,4,1.0,3,1.0,1.0,1.0,2,22,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,5.0,0.0,0.0,0.0,7.0,59,19.666667,19.666667,2,1,1.0,0.045455,1.000000,0.500000,2,2


(18371, 46)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,item_category_split_count,item_property_split_count,predict_category_split_count,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,pred_category_split_count,pred_property_split_count
154611,7,6,8,14,0,1002,2005,3001,4,13,0.985244,12,0.964023,0.966550,0.955378,2,23,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,6.0,9.0,0.0,0.0,0.0,22.084217,118,16.857143,3.371429,2,1,1.0,0.043478,0.500000,1.00,4,1
154612,6,9,8,16,1,1004,2002,3006,0,14,0.989068,13,0.963794,0.964225,0.959134,2,34,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,18.0,29.0,23.0,0.0,1.0,2.0,23.929679,137,22.833333,22.833333,2,0,1.0,0.000000,0.333333,0.00,6,2
154613,6,9,8,16,0,1004,2002,3004,10,14,0.989068,13,0.963794,0.964225,0.959134,2,34,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,29.0,83.0,0.0,1.0,1.0,23.929679,137,22.833333,2.075758,2,1,1.0,0.029412,0.333333,0.50,6,2
154614,6,9,8,16,1,1003,2002,3007,0,14,0.989068,13,0.963794,0.964225,0.959134,2,34,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,39.0,23.0,0.0,0.0,2.0,23.929679,137,22.833333,22.833333,2,1,1.0,0.029412,0.333333,1.00,6,1
154615,6,9,8,16,0,1002,2003,3002,0,14,0.989068,13,0.963794,0.964225,0.959134,2,34,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,14.0,83.0,0.0,1.0,1.0,23.929679,137,22.833333,22.833333,2,1,1.0,0.029412,0.285714,0.25,7,4


In [7]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

In [9]:
y_train = train_kflod[target]
x_train = train_kflod[predictors]

y_val = val_kflod[target]
x_val = val_kflod[predictors]

del train_kflod
del val_kflod

In [12]:
#train/val
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.1,
    'max_depth': 7,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'scale_pos_weight': 50,
    'seed': 201703,
    'missing': -1
}

xgbtrain = xgb.DMatrix(x_train, y_train)
xgbval = xgb.DMatrix(x_val)
model = xgb.train(params, xgbtrain, num_boost_round=1200)
y_pred = model.predict(xgbval)
y_pred = pd.Series(y_pred, name='val')

print(log_loss(y_val, y_pred))

0.170601342569


In [73]:
#train/test
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.1,
    'max_depth': 7,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'scale_pos_weight': 50,
    'seed': 201703,
    'missing': -1
}

xgbtrain = xgb.DMatrix(train[predictors], train[target])
xgbtest = xgb.DMatrix(test[predictors])
model = xgb.train(params, xgbtrain, num_boost_round=1200)
y_pred = model.predict(xgbtest)
y_pred = pd.Series(y_pred, name='predicted_score')

In [99]:
result = pd.read_csv('result.txt', sep=' ')

result.shape
result.head()

(36742, 2)

,0,1
0,NaN,0.016020
1,NaN,0.066969
2,NaN,0.141327
3,NaN,0.187762
4,NaN,0.041659


In [1]:
test.index = range(0, test.shape[0])

result = pd.concat([test.instance_id, y_pred], axis=1, ignore_index=True)
result.to_csv(
    'result.txt',
    sep=' ',
    header=['instance_id', 'predicted_score'],
    index=None)

NameError: name 'test' is not defined

# XGB 无历史数据

In [11]:
df = pd.read_csv('df_without_his.csv')

df.shape
df.head()

(478111, 52)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,user_query_day,user_query_day_hour,shop_score,item_score,item_score2,item_score3,predict_category_property_split,predict_category_split,predict_property_split,item_category_split,item_property_split,item_category_split_count,item_property_split_count,predict_category_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,1003,2005,3003,282924576738839389,1537236544,5,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 10:09:04,18,10,3,3,7.0,59,19.666667,3.277778,"['5799347067982556520:-1', '509660095530134768...","['5799347067982556520', '509660095530134768', ...",['9148482949976129397'],"['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,5,5,1,2,0,1.0,0.000000,0.400000,0.000000
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,1002,2005,3006,4007979028023783431,1537243232,0,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 12:00:32,18,12,8,5,7.0,59,19.666667,19.666667,"['5799347067982556520:9172976955054793469', '7...","['5799347067982556520', '7908382889764677758']","['9172976955054793469', '1787573075717641245,9...","['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,2,2,2,2,1,1.0,0.045455,1.000000,0.500000
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,0,1003,2005,3004,4884875192608989870,1537211052,0,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 03:04:12,18,3,2,2,7.0,59,19.666667,19.666667,"['5799347067982556520:5131280576272319091', '7...","['5799347067982556520', '7258015885215914736',...","['5131280576272319091', '5131280576272319091',...","['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,3,3,3,2,1,1.0,0.045455,0.666667,0.333333
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2681414445369714628,1,1004,2005,3006,840119421106178602,1537222670,15,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,3,1.0,1.0,1.0,0,2018-09-18 06:17:50,18,6,1,1,7.0,59,19.666667,1.229167,"['509660095530134768:-1', '5799347067982556520...","['509660095530134768', '5799347067982556520', ...","['1787573075717641245,9148482949976129397', '9...","['7908382889764677758', '5799347067982556520']","['2072967855524022579', '5131280576272319091',...",2,22,5,5,3,2,0,1.0,0.000000,0.400000,0.000000
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2729475788342039013,0,1002,2005,3001,1736769971710354684,1537271320,0,57993

In [16]:
predictors = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description', 'day', 'hour',
    'user_query_day', 'user_query_day_hour', 'shop_score', 'item_score',
    'item_score2', 'item_score3', 'item_category_split_count',
    'item_property_split_count', 'pred_category_split_count',
    'pred_property_split_count', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%'
]

target = ['is_trade']

In [18]:
y_train = df[df.day != 24][target]
x_train = df[df.day != 24][predictors]

y_val = df[df.day == 24][target]
x_val = df[df.day == 24][predictors]

x_train.shape
x_train.head()
x_val.shape
x_val.head()

del df

(420693, 33)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,day,hour,user_query_day,user_query_day_hour,shop_score,item_score,item_score2,item_score3,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%
0,3,3,4,14,1,1003,2005,3003,5,4,1.0,3,1.0,1.0,1.0,18,10,3,3,7.0,59,19.666667,3.277778,2,22,5,1,2,0,1.0,0.000000,0.400000,0.000000
1,3,3,4,14,0,1002,2005,3006,0,4,1.0,3,1.0,1.0,1.0,18,12,8,5,7.0,59,19.666667,19.666667,2,22,2,2,2,1,1.0,0.045455,1.000000,0.500000
2,3,3,4,14,0,1003,2005,3004,0,4,1.0,3,1.0,1.0,1.0,18,3,2,2,7.0,59,19.666667,19.666667,2,22,3,3,2,1,1.0,0.045455,0.666667,0.333333
3,3,3,4,14,1,1004,2005,3006,15,4,1.0,3,1.0,1.0,1.0,18,6,1,1,7.0,59,19.666667,1.229167,2,22,5,3,2,0,1.0,0.000000,0.400000,0.000000
4,3,3,4,14,0,1002,2005,3001,0,4,1.0,3,1.0,1.0,1.0,18,19,2,2,7.0,59,19.666667,19.666667,2,22,2,2,2,1,1.0,0.045455,1.000000,0.500000


(57418, 33)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,day,hour,user_query_day,user_query_day_hour,shop_score,item_score,item_score2,item_score3,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%
420693,7,1,4,9,1,1003,2005,3003,0,12,0.981128,10,0.974508,0.977220,0.966373,24,21,1,1,20.037886,37,5.285714,5.285714,2,14,3,3,2,1,1.0,0.071429,0.666667,0.333333
420694,7,1,4,9,0,1003,2002,3006,0,12,0.981128,10,0.974508,0.977220,0.966373,24,21,4,4,20.037886,37,5.285714,5.285714,2,14,9,1,2,0,1.0,0.000000,0.222222,0.000000
420695,8,12,14,18,0,1003,2002,3002,0,16,1.000000,15,0.962925,0.960578,0.983604,24,19,2,2,28.203749,264,33.000000,33.000000,2,49,5,4,2,0,1.0,0.000000,0.400000,0.000000
420696,8,12,14,18,0,1004,2005,3004,0,16,1.000000,15,0.962925,0.960578,0.983604,24,18,2,2,28.203749,264,33.000000,33.000000,2,49,7,3,2,1,1.0,0.020408,0.285714,0.333333
420697,8,12,14,18,1,1003,2002,3002,0,16,1.000000,15,0.962925,0.960578,0.983604,24,19,1,1,28.203749,264,33.000000,33.000000,2,49,6,4,2,0,1.0,0.000000,0.333333,0.000000


In [19]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

In [ ]:
#train/val
params = {
    'objective': 'binary:logistic',
    'eta': 0.07,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.1,
    'max_depth': 7,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'scale_pos_weight': 50,
    'seed': 201703,
    'missing': -1
}

xgbtrain = xgb.DMatrix(x_train, y_train)
xgbval = xgb.DMatrix(x_val)
model = xgb.train(params, xgbtrain, num_boost_round=3500)
y_pred = model.predict(xgbval)
y_pred = pd.Series(y_pred, name='val')

print(log_loss(y_val, y_pred))
xgb.plot_importance(model)